In [1]:
println("================================================================================================")

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.132:4040
SparkContext available as 'sc' (version = 3.5.0, master = local[*], app id = local-1699855239979)
SparkSession available as 'spark'


In [46]:
import scala.collection.mutable
import scala.util.Try

import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Column
import org.apache.spark.sql.types._

import scala.collection.mutable
import scala.util.Try
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Column
import org.apache.spark.sql.types._


In [3]:
// TODO FIX RUN NB INTO ANOTHER
//%run ./OSUtils.ipynb

def isWindows: Boolean = System.getProperty("os.name").toUpperCase.contains("WINDOWS")
def isLinux: Boolean = System.getProperty("os.name").toUpperCase.contains("LINUX")

def printOS: Unit =    
    if (isWindows) println("THIS IS A WINDOWS OS")
    else println("THIS IS A LINUX OS")

isWindows: Boolean
isLinux: Boolean
printOS: Unit


In [4]:
val dfFlights = spark.read.option("header", true).csv("./data/flights.csv")

dfFlights: org.apache.spark.sql.DataFrame = [DayofMonth: string, DayOfWeek: string ... 5 more fields]


In [5]:
dfFlights.printSchema

root
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- Carrier: string (nullable = true)
 |-- OriginAirportID: string (nullable = true)
 |-- DestAirportID: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- ArrDelay: string (nullable = true)



In [6]:
/** Check if schema from a df contains some values.
  * @param schema: Df schema
  * @param fieldsToCheck: List of columns to check
  * @return: Unit
  */
def schemaContainsField(schema: List[String], fieldsToCheck: List[String]): Unit = {
    fieldsToCheck.foreach(f => {
        if (schema.contains(f)) println(s"$f --> true")
        else println(s"$f --> false")})
}

schemaContainsField: (schema: List[String], fieldsToCheck: List[String])Unit


In [7]:
val fieldsToCheck: List[String] = List("DayofMonth", "Carrier", "carrier") // Case sensitive!!

fieldsToCheck: List[String] = List(DayofMonth, Carrier, carrier)


In [8]:
schemaContainsField(dfFlights.schema.names.toList, fieldsToCheck)

DayofMonth --> true
Carrier --> true
carrier --> false


In [9]:
/** Check if df has single column
  * @param df: DataFrame
  * @param column:Column to check
  * @return: Boolean if df has column
*/
def hasColumn(df: DataFrame, column: String): Boolean = Try(df(column)).isSuccess

hasColumn: (df: org.apache.spark.sql.DataFrame, column: String)Boolean


In [10]:
hasColumn(dfFlights, "DayOfWeek")

res3: Boolean = true


In [11]:
hasColumn(dfFlights, "Aeroport")

res4: Boolean = false


In [12]:
def addSufixColumns(df: DataFrame, suf: String): DataFrame = {
    val origColumns = df.columns
    val colsRenamed = origColumns.map {_ + suf}
    df.toDF(colsRenamed:_*)
}

addSufixColumns: (df: org.apache.spark.sql.DataFrame, suf: String)org.apache.spark.sql.DataFrame


In [14]:
val newDf = addSufixColumns(dfFlights, "_FLGHTS")

newDf: org.apache.spark.sql.DataFrame = [DayofMonth_FLGHTS: string, DayOfWeek_FLGHTS: string ... 5 more fields]


In [15]:
newDf.printSchema

root
 |-- DayofMonth_FLGHTS: string (nullable = true)
 |-- DayOfWeek_FLGHTS: string (nullable = true)
 |-- Carrier_FLGHTS: string (nullable = true)
 |-- OriginAirportID_FLGHTS: string (nullable = true)
 |-- DestAirportID_FLGHTS: string (nullable = true)
 |-- DepDelay_FLGHTS: string (nullable = true)
 |-- ArrDelay_FLGHTS: string (nullable = true)



In [18]:
/**
   * Add columns to dataframe
   * @param df dataframe
   * @param col pair of name and column (name,column)
   * @return df with new columns
   */
def addColumn(df: DataFrame, col: (String, Column)*): DataFrame =
    col.foldLeft(df) { case (dfbase, (name, col)) => dfbase.withColumn(name, col) }

addColumn: (df: org.apache.spark.sql.DataFrame, col: (String, org.apache.spark.sql.Column)*)org.apache.spark.sql.DataFrame


In [27]:
val dfAdded = addColumn(dfFlights, ("LITERAL", lit("XXX")))

dfAdded: org.apache.spark.sql.DataFrame = [DayofMonth: string, DayOfWeek: string ... 6 more fields]


In [28]:
dfAdded.printSchema

root
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- Carrier: string (nullable = true)
 |-- OriginAirportID: string (nullable = true)
 |-- DestAirportID: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- LITERAL: string (nullable = false)



In [29]:
dfAdded.show(5, false)

+----------+---------+-------+---------------+-------------+--------+--------+-------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|LITERAL|
+----------+---------+-------+---------------+-------------+--------+--------+-------+
|19        |5        |DL     |11433          |13303        |-3      |1       |XXX    |
|19        |5        |DL     |14869          |12478        |0       |-8      |XXX    |
|19        |5        |DL     |14057          |14869        |-4      |-15     |XXX    |
|19        |5        |DL     |15016          |11433        |28      |24      |XXX    |
|19        |5        |DL     |11193          |12892        |-6      |-11     |XXX    |
+----------+---------+-------+---------------+-------------+--------+--------+-------+
only showing top 5 rows



In [30]:
/**
    *
    * @param df
    * @param columns
    * @return
    */
def findDuplicateRows(df: DataFrame, columns: Seq[String]): DataFrame = {
  df.groupBy(columns.head,columns.tail:_*)
    .agg(count("*").alias("cnt"))
    .filter(col("cnt") > 1)
    .select(columns.map(col): _*)
}

findDuplicateRows: (df: org.apache.spark.sql.DataFrame, columns: Seq[String])org.apache.spark.sql.DataFrame


In [32]:
findDuplicateRows(dfFlights, dfFlights.schema.names.toSeq).show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        11|        4|     AA|          13930|        12953|       0|       0|
|        17|        3|     MQ|          12451|        13303|       0|       0|
|        21|        7|     OO|          14831|        12892|       0|       0|
|        23|        2|     EV|          11618|        10693|       0|       0|
|         4|        4|     AA|          11298|        13495|       0|       0|
|        11|        4|     AA|          13930|        14747|       0|       0|
|        11|        4|     MQ|          11193|        13930|       0|       0|
|        16|        2|     AA|          10721|        11298|       0|       0|
|        16|        2|     AA|          14869|        11298|       0|       0|
|        17|        3|     EV|          13931|      

In [33]:
/**
*
* @param c
* @return
*/
def stringify(c: Column) = concat(lit("["), concat_ws(",", c), lit("]"))

stringify: (c: org.apache.spark.sql.Column)org.apache.spark.sql.Column


In [36]:
val dfStringify = dfFlights.withColumn("colStringified", stringify(dfFlights.col("DayofMonth")))

dfStringify: org.apache.spark.sql.DataFrame = [DayofMonth: string, DayOfWeek: string ... 6 more fields]


In [37]:
dfStringify.show()

+----------+---------+-------+---------------+-------------+--------+--------+--------------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|colStringified|
+----------+---------+-------+---------------+-------------+--------+--------+--------------+
|        19|        5|     DL|          11433|        13303|      -3|       1|          [19]|
|        19|        5|     DL|          14869|        12478|       0|      -8|          [19]|
|        19|        5|     DL|          14057|        14869|      -4|     -15|          [19]|
|        19|        5|     DL|          15016|        11433|      28|      24|          [19]|
|        19|        5|     DL|          11193|        12892|      -6|     -11|          [19]|
|        19|        5|     DL|          10397|        15016|      -1|     -19|          [19]|
|        19|        5|     DL|          15016|        10397|       0|      -1|          [19]|
|        19|        5|     DL|          10397|        14869|

In [42]:
val schemaArr = dfFlights.schema.toArray

schemaArr: Array[org.apache.spark.sql.types.StructField] = Array(StructField(DayofMonth,StringType,true), StructField(DayOfWeek,StringType,true), StructField(Carrier,StringType,true), StructField(OriginAirportID,StringType,true), StructField(DestAirportID,StringType,true), StructField(DepDelay,StringType,true), StructField(ArrDelay,StringType,true))


In [44]:
schemaArr(1)

res21: org.apache.spark.sql.types.StructField = StructField(DayOfWeek,StringType,true)


In [47]:
case class STFieldCC(colName: String, field: StructField)

defined class STFieldCC


In [48]:
def checkIfSameSchema(df1: DataFrame, df2: DataFrame): Boolean = df1.schema.equals(df2.schema)

checkIfSameSchema: (df1: org.apache.spark.sql.DataFrame, df2: org.apache.spark.sql.DataFrame)Boolean


In [58]:
dfFlights.printSchema

root
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- Carrier: string (nullable = true)
 |-- OriginAirportID: string (nullable = true)
 |-- DestAirportID: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- ArrDelay: string (nullable = true)



In [60]:
val dfFlightsLong = dfFlights.withColumn("LONG_COL", lit(1))

dfFlightsLong: org.apache.spark.sql.DataFrame = [DayofMonth: string, DayOfWeek: string ... 6 more fields]


In [62]:
dfFlightsLong.printSchema()

root
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- Carrier: string (nullable = true)
 |-- OriginAirportID: string (nullable = true)
 |-- DestAirportID: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- LONG_COL: integer (nullable = false)



In [67]:
/**
    *
    * @param df
    * @param fieldName
    * @param dataType
    * @return
    */
def castColumnTo(df: DataFrame, fieldName: String, dataType: DataType ) : DataFrame = 
    df.withColumn( fieldName, df(fieldName).cast(dataType))

castColumnTo: (df: org.apache.spark.sql.DataFrame, fieldName: String, dataType: org.apache.spark.sql.types.DataType)org.apache.spark.sql.DataFrame


In [68]:
val dfFlightsString = castColumnTo(dfFlightsLong, "LONG_COL", StringType)

dfFlightsString: org.apache.spark.sql.DataFrame = [DayofMonth: string, DayOfWeek: string ... 6 more fields]


In [69]:
dfFlightsString.printSchema()

root
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- Carrier: string (nullable = true)
 |-- OriginAirportID: string (nullable = true)
 |-- DestAirportID: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- LONG_COL: string (nullable = false)



In [70]:
checkIfSameSchema(dfFlightsLong, dfFlightsString)

res37: Boolean = false
